In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": 0},
    attn_implementation="sdpa",
)
# model = model.eval()
model.requires_grad_(False)

from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    # "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    # test_size=250,
    test_size=350,
    # test_size=999,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)

correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

print(len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset))

correct_dataset = correct_dataset[:30]
len(correct_dataset)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
224 202 0.9017857142857143


30

## Обучение по чанкам в цикле

In [ ]:
import torch

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm
from tqdm import tqdm as text_tqdm
from hidden_capacity_reasoning.utils import (
    tokenize_single_turn,
    EOS_TOKEN_ID,
    END_THINK_ID,
)

torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

max_new_tokens = 200
compression_tokens_amount = 4
max_total_tokens = 4096
max_total_steps = max_total_tokens // max_new_tokens + 1

evaluation_dataset = []
correct_items = 0
model.generation_config.pad_token_id = tokenizer.pad_token_id

for dataset_pos in tqdm(range(len(correct_dataset))):
# for dataset_pos in tqdm(range(1, len(correct_dataset))):
    tokenized_turn = tokenize_single_turn(
        question=base_prompt.format(question=correct_dataset[dataset_pos]["problem"]),
        answer=correct_dataset[dataset_pos]["model_answer"],
        tokenizer=tokenizer,
    )
    for key in tokenized_turn.keys():
        tokenized_turn[key] = torch.tensor(tokenized_turn[key])

    device = "cuda"

    content_compression_mask = tokenized_turn["content_compression_mask"]

    input_part_end = (content_compression_mask == 0).nonzero()[-3][0]
    # get only question part
    question_input_ids = (
        tokenized_turn["input_ids"][: int(input_part_end) + 1].unsqueeze(0).cuda()
    )
    # print(tokenizer.decode(question_input_ids[-1]))

    ######## start loop generation
    ########
    compression_loop = True
    input_ids_embeds = model.get_input_embeddings()(question_input_ids)
    compression_part = torch.tensor([[0]])
    generated_ids_new = None
    generated_embeds = None
    generated_embeds_prev = None
    generated_ids_new_prev = None
    end_of_think = False
    total_generated_text = ""

    for compression_step in text_tqdm(range(max_total_steps)):
        ######## generate new tokens
        ########
        inputs_embeds = None
        with torch.no_grad():

            if compression_part.shape[1] >= compression_tokens_amount:
                generated_embeds_prev = generated_embeds[
                    :, -(max_new_tokens // 2) :, :
                ].clone()
                inputs_embeds = torch.cat(
                    [
                        input_ids_embeds,
                        compression_part,
                        generated_embeds_prev,
                    ],
                    dim=1,
                )
            else:
                # first time generation
                inputs_embeds = torch.cat(
                    [
                        input_ids_embeds,
                    ],
                    dim=1,
                )
            generated_ids_new = model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=torch.ones(
                    inputs_embeds.shape[:2],
                    device="cuda",
                ).long(),
                max_new_tokens=max_new_tokens,
                do_sample=False,
                use_cache=compression_step > 0,
            )
            # break
        generated_result = tokenizer.decode(generated_ids_new[-1])
        # print(generated_result)
        total_generated_text += generated_result
        print("=" * 50)
        generated_embeds = model.get_input_embeddings()(generated_ids_new)
        if END_THINK_ID in generated_ids_new[-1].tolist():
            end_of_think = True
            break

        ########
        ######## get original language loss
        ########
        labels = None
        if compression_part.shape[1] >= compression_tokens_amount:
            labels = torch.cat(
                [
                    question_input_ids.cuda(),
                    ((torch.ones(compression_part.shape[:2]) * -100).long()).cuda(),
                    (
                        (torch.ones(generated_embeds_prev.shape[:2]) * -100).long()
                    ).cuda(),
                    generated_ids_new.cuda(),
                ],
                dim=1,
            )
        else:
            # first time generation
            labels = torch.cat(
                [
                    question_input_ids.cuda(),
                    generated_ids_new.cuda(),
                ],
                dim=1,
            )

        question_content_mask = content_compression_mask[
            : int(input_part_end) + 1
        ].clone()
        question_content_mask[question_content_mask == 0] = 4
        question_content_mask[question_content_mask == 1] = 0
        question_content_mask[question_content_mask == 4] = 1

        if compression_part.shape[1] >= compression_tokens_amount:
            train_content_mask_new = torch.cat(
                [
                    question_content_mask,
                    torch.zeros(compression_part.shape[1]),
                    torch.zeros(generated_embeds_prev.shape[1]),
                    torch.zeros(generated_ids_new.shape[1] // 2),
                    torch.ones(generated_ids_new.shape[1] // 2),
                ]
            ).long()
        else:
            train_content_mask_new = torch.cat(
                [
                    question_content_mask,
                    torch.ones(generated_ids_new.shape[1] // 2) * 0,
                    torch.ones(generated_ids_new.shape[1] // 2),
                ]
            ).long()

        generated_ids_new_prev = generated_ids_new.clone()
        # generated_embeds = model.get_input_embeddings()(generated_ids_new)

        new_input_embeds = None
        if compression_part.shape[1] >= compression_tokens_amount:
            new_input_embeds = torch.cat(
                [
                    input_ids_embeds.cuda(),
                    compression_part.cuda(),
                    generated_embeds_prev.cuda(),
                    generated_embeds,
                ],
                dim=1,
            )
        else:
            new_input_embeds = torch.cat(
                [
                    input_ids_embeds,
                    generated_embeds,
                ],
                dim=1,
            )

        labels[:, train_content_mask_new == 0] = -100

        with torch.no_grad():
            original_loss = model(
                inputs_embeds=new_input_embeds,
                labels=labels,
            ).loss
        print("original_loss", original_loss)
        ########
        ######## generate compress embeddings
        ########
        compression_tensor = torch.nn.Parameter(
            torch.rand_like(
                new_input_embeds[:, :compression_tokens_amount, :],
            )
            * model.get_input_embeddings().weight.data.std(),
            requires_grad=True,
        )

        question_labels = question_input_ids.clone().cuda()
        question_labels[0][question_content_mask == 0] = -100
        compression_tensor_labels = (
            (torch.ones(compression_tensor.shape[:2]) * -100).long().cuda()
        )
        if compression_part.shape[1] >= compression_tokens_amount:
            compression_part_labels = (
                (torch.ones(compression_part.shape[:2]) * -100).long().cuda()
            )
            compressed_labels = torch.cat(
                [
                    question_labels,
                    compression_part_labels,
                    compression_tensor_labels,
                    generated_ids_new[:, -(max_new_tokens // 2) :],
                ],
                dim=-1,
            )
        else:
            compressed_labels = torch.cat(
                [
                    question_labels,
                    compression_tensor_labels,
                    generated_ids_new[:, -(max_new_tokens // 2) :],
                ],
                dim=-1,
            )

        ########
        ######## train
        ########
        epoch_amount = 100

        optimizer = torch.optim.Adam([compression_tensor], lr=0.1)
        acclumulation_steps = 1
        for epoch in range(epoch_amount):
            if compression_part.shape[1] >= compression_tokens_amount:
                compressed_inputs_embeds = torch.cat(
                    [
                        input_ids_embeds.detach(),
                        compression_part.detach(),
                        compression_tensor,
                        generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
                    ],
                    dim=1,
                )
            else:
                compressed_inputs_embeds = torch.cat(
                    [
                        input_ids_embeds.detach(),
                        compression_tensor,
                        generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
                    ],
                    dim=1,
                )
            compression_loss = model(
                inputs_embeds=compressed_inputs_embeds,
                labels=compressed_labels,
            ).loss
            compression_loss.backward()
            if (epoch + 1) % acclumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            if compression_loss.item() <= original_loss.item():
                break
            # if compression_loss.item() <= (original_loss.item() + 0.01):
            #     break
        print("compression_loss", compression_loss)
        if compression_part.shape[1] >= compression_tokens_amount:
            compression_part = torch.cat(
                [
                    compression_part,
                    compression_tensor.detach(),
                ],
                dim=1,
            )
        else:
            compression_part = compression_tensor.detach()
    inputs_embeds = torch.cat(
        [
            inputs_embeds,
            generated_embeds,
        ],
        dim=1,
    )
    final_response = model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=torch.ones(
            inputs_embeds.shape[:2],
            device="cuda",
        ).long(),
        max_new_tokens=(max_total_steps - compression_step) * max_new_tokens,
        do_sample=False,
    )
    final_answer = tokenizer.decode(final_response[-1])
    total_generated_text += final_answer
    # print("FINAL ANSWER", final_answer)

    gold_answer = correct_dataset[dataset_pos]["answer"]
    answer = dataset_answer_filter(gold_answer)
    # print("GOLD ANSWER", answer)
    model_answer = model_answer_filter(total_generated_text)
    if is_equiv(answer, model_answer):
        correct_items += 1
        print("CORRECT")
    else:
        print("WRONG", gold_answer)
        print(total_generated_text)
        
    compressed_total_len = inputs_embeds.shape[1] + final_response.shape[1]
    total_generated_tokens = final_response.shape[1] + max_new_tokens * (
        compression_step + 1
    )
    original_total_len = len(
        tokenizer.encode(
            correct_dataset[dataset_pos]["model_answer"],
            add_special_tokens=False,
        )
    )
    print(
        f"вопрос+сжатые+сгенерированные={compressed_total_len}, всего_сгенерированно_токенов={total_generated_tokens} оригинальная_генерация={original_total_len}"
    )
    evaluation_dataset.append(
        {
            "original_total_len": original_total_len,
            "compressed_total_len": compressed_total_len,
        }
    )
    # break

  0%|          | 0/30 [00:00<?, ?it/s]

/home/user-name-goes-here/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/user-name-goes-here/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


original_loss tensor(0.4852, device='cuda:0')


compression_loss tensor(0.4828, device='cuda:0', grad_fn=<NllLossBackward0>)
original_loss tensor(0.3640, device='cuda:0')


compression_loss tensor(0.4036, device='cuda:0', grad_fn=<NllLossBackward0>)
original_loss tensor(0.3844, device='cuda:0')


...

compression_loss tensor(0.4689, device='cuda:0', grad_fn=<NllLossBackward0>)
original_loss tensor(0.5037, device='cuda:0')


compression_loss tensor(0.5023, device='cuda:0', grad_fn=<NllLossBackward0>)
original_loss tensor(0.5118, device='cuda:0')


compression_loss tensor(0.5116, device='cuda:0', grad_fn=<NllLossBackward0>)
original_loss tensor(0.4416, device='cuda:0')


compression_loss tensor(0.4333, device='cuda:0', grad_fn=<NllLossBackward0>)


 24%|██▍       | 5/21 [00:32<01:43,  6.44s/it]

In [ ]:
# 